<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/Extract_WebData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year


---
This script extracts the queue information where it primarily consists of Urls, we will check if the Url is valid then extract the details

Libraries Used:



*   Beautiful soup
*   Regex
*   UrlLib
*   Regex

Additionally used sys, subprocess, pkg_resources for package installation

Install all the  Packages used in the project if missing

In [10]:

#Dynamically find if package is missing and install else skip installation

import sys
import subprocess
from traceback import format_exc
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables
!pip3 install urlparse


ERROR: Could not find a version that satisfies the requirement azure-storages (from versions: none)
ERROR: No matching distribution found for azure-storages
ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [11]:

import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')

In [12]:
webPageReadTimeout = 10 #For soup object


Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md



In [13]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
crawled_queue_name = "queue-crawledarchiveurls"
extractedDetails_queue_name = "queue-extractedpagedetails"


queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode


In [14]:

def QueueUrlContents(jsonData):
  print("Queueing data: "+jsonData)
  queue_service.put_message(extractedDetails_queue_name,jsonData)



#Fetch the data from web pages via Beautiful soup


In [15]:

from bs4 import BeautifulSoup
from urllib.request import urlopen


def GetSoupContent(url):
  try:
    if url != None:
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      divEle = soup.find(id = "content")
      return divEle.text
  except:
    print("Problem getting text content. url: "+url)
  return None

def getSoupObj(url):
  try:
    if url != None:
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      print("getSoupObj",url)
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)
  return None

Queue Details with extracted info

In [16]:

def ExisitsInArray(arrDet, valToChk):
  try:
    return arrDet.index(valToChk) >= 0
  except: 
    return False
  return False


In [17]:
import json

allLinks = []
queueMessages = [] #queue messages
queueUrlCount = 0
def LoopThroughUrls():
  try:
    metadata = queue_service.get_queue_metadata(crawled_queue_name)
    queueUrlCount = metadata.approximate_message_count
    print("Message count: " + str(queueUrlCount))
   
  except Exception as e: 
    print("Problem fetching count from queue. Message : "+ str(e)) 
    return None   

  try:   

    queueMessages = queue_service.get_messages(crawled_queue_name)
    while queueMessages != None and len(queueMessages) > 0:
      for urlMsg in queueMessages:
        if urlMsg != None:
          url = urlMsg.content 
          queue_service.delete_message(crawled_queue_name,urlMsg.id, urlMsg.pop_receipt)
          if ExisitsInArray(allLinks,url) == False: #Check if the Url is not already added to the list
            allLinks.append(url)
            #Queue new data
            txt = GetSoupContent(url)
            if txt != None:
              txt= txt.strip()
              jsonData = {"Url":url, "TextInfo":txt}
              json_dump = json.dumps(jsonData)
              print(json_dump)
              QueueUrlContents(json_dump)
      queueMessages = queue_service.get_messages(crawled_queue_name)      
            
  except Exception as e: 
    print("Problem processing urls. Message : "+ str(e))
    return None

In [18]:

from datetime import datetime

if __name__ == '__main__':
  print("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
  LoopThroughUrls()
  print("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))

initiaing crawling: 29/04/2022 04:02:20
Message count: 0
End of crawling: 29/04/2022 09:32:21
